<a href="https://colab.research.google.com/github/aliciawill/BigData/blob/master/%EC%98%81%ED%99%94%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_surprise%EC%9D%B4%EC%9A%A9-%EC%88%98%EC%97%85%EC%A4%91%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 5.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619406 sha256=4b5f3f35bfe14e534e356ecaa052381483d8a04c834a9622e19f7807599fab2c
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
import surprise

In [5]:
print(surprise.__version__)

1.1.1


In [24]:
from surprise import SVD 
## 고유값분해(행렬을 더 낮은 차원으로 분해)해서 다시 원본으로 살리면서 잠재적인 값을 추론하는 방법
from surprise import Dataset, Reader ## SVD를 사용하기 위한 데이터셋 만들어주는 클래스
                            ## 사용자, 아이템, 평점
from surprise import accuracy ## RMSE, MAE, cross-validation(k-fold)
from surprise.model_selection import train_test_split ## 훈련/검증 데이터 분류

In [12]:
## 1. 데이터셋을 만들어주자(사용자, 아이템, 평점),  훈련/검증 데이터 분류
data = Dataset.load_builtin('ml-100k')
data

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [13]:
trainset, testset = train_test_split(data, test_size= 0.25, random_state=0)

In [14]:
trainset

In [16]:
testset[:5]

[('120', '282', 4.0),
 ('882', '291', 4.0),
 ('535', '507', 5.0),
 ('697', '244', 5.0),
 ('751', '385', 4.0)]

In [17]:
## 2. SVD() 모델 선정
algo = SVD()

In [18]:
## 3. 훈련용 데이터로 fit()
algo.fit(trainset)

In [21]:
## 4. 검증용 데이터로 예측
predictions = algo.test(testset)[:5]
predictions 

[Prediction(uid='120', iid='282', r_ui=4.0, est=3.660035800131191, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.767923368425111, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.18063586219212, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8349670102796005, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.398638057559097, details={'was_impossible': False})]

In [22]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions]

[('120', '282', 3.660035800131191),
 ('882', '291', 3.767923368425111),
 ('535', '507', 4.18063586219212),
 ('697', '244', 3.8349670102796005),
 ('751', '385', 3.398638057559097)]

In [ ]:
## 5. 정확도 계산 

In [23]:
## 6. 사용자, 아이템정보 주고 예측 실행
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
pred

Prediction(uid='196', iid='302', r_ui=None, est=4.426410838418583, details={'was_impossible': False})

In [25]:
import pandas as pd

In [26]:
## csv를 SVD하기 위한 DataSet를 주기 위해서는
## 1) df <--- csv, 2) DataSet <--- df

In [28]:
ratings = pd.read_csv('ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [29]:
reader = Reader(rating_scale=(0.5, 5.0))

In [30]:
## SVD에서 사용할 수 있는 데이터 셋으로 만들어주자.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data

In [31]:
## 데이터 분류, 객체생성, 훈련시키고, 검증해보고, 검증결과 프린트

In [41]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [48]:
algo = SVD(n_factors=50, n_epochs= 40,  random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8784


0.8783983314109463

In [44]:
from surprise.model_selection import cross_validate

In [45]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8712  0.8692  0.8790  0.8719  0.8662  0.8715  0.0042  
MAE (testset)     0.6701  0.6694  0.6736  0.6709  0.6659  0.6700  0.0025  
Fit time          3.22    3.22    3.20    3.25    3.19    3.21    0.02    
Test time         0.20    0.16    0.16    0.16    0.15    0.16    0.02    


{'fit_time': (3.217305898666382,
  3.2187728881835938,
  3.1975338459014893,
  3.2546420097351074,
  3.1863694190979004),
 'test_mae': array([0.67007849, 0.66944509, 0.67355579, 0.67093258, 0.66590802]),
 'test_rmse': array([0.87120224, 0.8691832 , 0.8789537 , 0.87191334, 0.86623365]),
 'test_time': (0.20156240463256836,
  0.15647292137145996,
  0.15547537803649902,
  0.1588599681854248,
  0.14675617218017578)}

In [33]:
import time

In [35]:
start = time.time()
start

1640231315.354797

In [36]:
end = time.time()
end

1640231323.7985976

In [37]:
end-start

8.443800687789917

In [38]:
end2 = time.time()
end2

1640231344.1239731

In [39]:
end2-start

28.769176244735718

In [52]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [53]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.50   {'was_impossible': False}


In [54]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [55]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Toy Story (1995) : 3.5002644558160445
Jumanji (1995) : 3.5002644558160445
Grumpier Old Men (1995) : 3.5002644558160445
Waiting to Exhale (1995) : 3.5002644558160445
Father of the Bride Part II (1995) : 3.5002644558160445
Heat (1995) : 3.5002644558160445
Sabrina (1995) : 3.5002644558160445
Tom and Huck (1995) : 3.5002644558160445
Sudden Death (1995) : 3.5002644558160445
GoldenEye (1995) : 3.5002644558160445
